# Assignment 2 

Prediction of Airbnb apartments prices in New York city 

### Getting the dataset

In [1]:
import os
import sys
import warnings
from datetime import datetime
import numpy as np
import pandas as pd
import regex as re
import statsmodels.api as sm
import statsmodels.formula.api as smf
from mizani.formatters import percent_format
from plotnine import *

warnings.filterwarnings("ignore")

In [2]:
current_path = os.getcwd()
dirname = current_path.split("Code")[0]
Data = dirname + "Data"
output = dirname + "Output"

In [3]:
# Import data
data = pd.read_csv(Data + "\\listings.csv", index_col=0)

In [4]:
data.shape

(41533, 74)

### Preparing the data

In [5]:
data.columns

Index(['listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar

In [6]:
data["n_days_since"] = (
    data.calendar_last_scraped.apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
    - data.first_review.fillna("1950-01-01").apply(
        lambda x: datetime.strptime(x, "%Y-%m-%d")
    )
).dt.days

data["n_days_since"] = np.where(data.first_review.isnull(), np.nan, data.n_days_since)

In [7]:
# Outcome variable
data["price"] = data.price.str.replace("\\$", "").str.replace(",","").astype(float)

In [8]:
# Restriction based on assignment - small apartments with accommodating 2-6 people 
data = data[ (data['accommodates'] >=2) & (data['accommodates'] <=6)]

In [9]:
data["host_response_time"] = data["host_response_time"].astype("category")

In [10]:
data["neighbourhood_cleansed"] = np.where(
    data["neighbourhood_cleansed"].isin(
        data["neighbourhood_cleansed"]
        .value_counts()
        .loc[lambda x: x < 75]
        .index.tolist()
    ),
    "Other",
    data["neighbourhood_cleansed"],
)
data["neighbourhood_cleansed"] = data["neighbourhood_cleansed"].astype("category")
data["neighbourhood_group_cleansed"] = data["neighbourhood_group_cleansed"].astype("category")

In [11]:
data["property_type"] = data.property_type.str.lower().str.replace("room ", "")
data["property_type"] = data.property_type.str.replace("shared ", "").str.replace("in ","")
data["property_type"] = data.property_type.str.replace("entire ", "").str.replace("private ","")
data['property_type'] = data['property_type'].str.replace(r'(^.*home.*$)', 'home')
data['property_type'] = data['property_type'].str.replace(r'(^.*hotel.*$)', 'hotel')
data['property_type'] = data['property_type'].str.replace(r'(^.*house.*$)', 'house')
data["property_type"] = data.property_type.str.replace("casa particular","bed and breakfast")

In [12]:
data = data.loc[
    lambda x: x["property_type"].isin(
        [
            "rental unit",
            "home",
            "loft",
            "condo",
            "serviced apartment",
            "guest suite",
            
        ]
    )
]
data["property_type"] = data["property_type"].astype("category")

In [13]:
data["reviews_per_month"].value_counts().head()

0.02    763
0.03    668
0.01    661
0.06    492
0.04    487
Name: reviews_per_month, dtype: int64

In [14]:
data = data[data["room_type"].str.contains("Hotel room") == False]
data["room_type"] = data["room_type"].astype("category")
data["room_type"] = data["room_type"].map(
    {
        "Entire home/apt": "Entire/Apt",
        "Private room": "Private",
        "Shared room": "Shared"
    }
)

In [15]:
for perc in ["host_response_rate", "host_acceptance_rate"]:
    data[perc] = pd.to_numeric(data[perc].str.replace("%",""), errors="coerce")

In [16]:
# Bathroom data cleaning 
data["bathrooms_text"] = data.bathrooms_text.str.replace("Half-bath", "0.5 bath").str.replace("half-bath","0.5 bath")

data['shared_bath'] = data["bathrooms_text"].map(lambda x: True if 'shared' in str(x) else False)

data["n_bathrooms"] = data.bathrooms_text.str.replace("Shared", "").str.replace("Private", "").str.split().str[0].apply(float)

In [17]:
# format binary variables
for binary in [
    "host_is_superhost",
    "host_has_profile_pic",
    "host_identity_verified",
    "instant_bookable",
    "has_availability",
]:
    data[binary] = data[binary].map({"t": True, "f": False})

In [18]:
data.drop(columns=['listing_url', 'scrape_id', 'last_scraped', 'source', 'name','description',
                   'neighborhood_overview', 'picture_url', 'host_id', 'host_url', 'host_name', 
                   'host_since', 'host_location', 'host_about', 'host_picture_url',
                   'host_neighbourhood', 'host_listings_count', 'host_verifications', 'neighbourhood',
                   'latitude', 'longitude', 'bathrooms', 'bathrooms_text', 'minimum_nights', 'maximum_nights', 
                   'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm',
                   'maximum_nights_avg_ntm', 'calendar_updated', 'availability_30', 'availability_60', 
                   'availability_90', 'availability_365', 'calendar_last_scraped', 'number_of_reviews_ltm', 
                   'first_review', 'last_review', 'license', 'calculated_host_listings_count',
                   'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms',
                   'calculated_host_listings_count_shared_rooms', 'host_thumbnail_url'], inplace=True)

In [19]:
data.describe()

,host_response_rate,host_acceptance_rate,host_total_listings_count,accommodates,bedrooms,beds,price,minimum_minimum_nights,number_of_reviews,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,n_days_since,n_bathrooms
count,19641.000000,20764.000000,30135.000000,30139.000000,27053.000000,29589.000000,30139.000000,30139.000000,30139.000000,30139.000000,24089.000000,23768.000000,23777.000000,23765.000000,23773.000000,23763.000000,23763.000000,24089.000000,24089.000000,30119.000000
mean,94.579197,84.097091,108.486146,2.944988,1.322108,1.576126,197.208467,18.369521,27.390723,0.718537,4.652776,4.771451,4.644463,4.826557,4.830698,4.747480,4.658885,1.284434,1259.188758,1.116886
std,15.102378,24.594476,644.710758,1.244842,0.594567,0.856347,919.464357,33.302998,55.180606,1.474757,0.694515,0.431702,0.518609,0.389258,0.397340,0.389383,0.461622,1.695721,1045.811142,0.349729
min,0.000000,0.000000,1.000000,2.000000,1.000000,1.000000,10.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000
25%,98.000000,79.000000,1.000000,2.000000,1.000000,1.000000,91.000000,2.000000,1.000000,0.000000,4.610000,4.727500,4.500000,4.810000,4.820000,4.660000,4.560000,0.140000,294.000000,1.000000
50%,100.000000,96.000000,2.000000,2.000000,1.000000,1.000000,141.000000,7.000000,6.000000,0.000000,4.830000,4.900000,4.800000,4.950000,4.960000,4.860000,4.770000,0.600000,1127.000000,1.000000
75%,100.000000,100.000000,5.000000,4.000000,2.000000,2.000000,215.000000,30.000000,27.000000,1.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,4.950000,1.950000,2044.000000,1.000000
max,100.000000,100.000000,7149.000000,6.000000,6.000000,12.000000,98159.000000,1250.000000,1666.000000,44.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,61.700000,4961.000000,6.000000


In [20]:
# Dummy from amenities
data["amenities"] = (
    data["amenities"]
    .str.strip("{}")
    .str.replace('"', "")
    .str.replace("[", "")
    .str.replace("]", "")
    .str.replace("–", "")
    .str.replace("-", "")
    .str.replace("\\u", "'", regex=False)
    .str.replace("\d", "", regex=True)
    .str.replace("'", "")
    .str.lower()
    .str.replace("mbps", "",regex=False)
    .str.strip()
    .str.split(",")
)

In [21]:
data['n_amenities'] = data["amenities"].apply(len)

In [22]:
amenities_keep = data["amenities"].explode().str.strip().value_counts().loc[lambda x: x>1000]
amenities_keep.to_pickle("amenities_to_keep.pkl")
amenities_to_keep = pd.read_pickle("amenities_to_keep.pkl")
# create dummy vars
dummies = amenities_to_keep.index.tolist()

for col in dummies:
    data[col] = data["amenities"].map(lambda x: True if col in str(x) else False)

In [23]:
to_filter = data.isna().sum()
to_filter[to_filter > 0]

host_response_time             10498
host_response_rate             10498
host_acceptance_rate            9375
host_total_listings_count          4
host_has_profile_pic               4
host_identity_verified             4
bedrooms                        3086
beds                             550
review_scores_rating            6050
review_scores_accuracy          6371
review_scores_cleanliness       6362
review_scores_checkin           6374
review_scores_communication     6366
review_scores_location          6376
review_scores_value             6376
reviews_per_month               6050
n_days_since                    6050
n_bathrooms                       20
dtype: int64

In [24]:
# 2. imput when few, not that important
data = data.assign(
    bedrooms=lambda x: x["bedrooms"].fillna(np.median(x["bedrooms"].dropna())),
    beds=lambda x: np.where(x["beds"].isnull(), x["accommodates"], x["beds"]),
    n_bathrooms=lambda x: x["n_bathrooms"].fillna(1),
    host_has_profile_pic=lambda x: x["host_has_profile_pic"].fillna(False),
    host_identity_verified=lambda x: x["host_identity_verified"].fillna(False),
    host_total_listings_count=lambda x: x["host_total_listings_count"].fillna(0),
)

In [25]:
# 3. drop columns when many missing not important
data = data.drop(["host_response_rate", "review_scores_location", "review_scores_communication", 
                 "review_scores_accuracy", "review_scores_cleanliness", "review_scores_checkin",
                 "host_acceptance_rate", "review_scores_value", 'amenities'], axis=1)

In [26]:
# 4. Replace missing variables re reviews with zero, when no review + add flags
data = data.assign(
    flag_days_since=np.multiply(data.n_days_since.isna(), 1),
    n_days_since=data.n_days_since.fillna(np.median(data.n_days_since.dropna())),
    flag_review_scores_rating=np.multiply(data.review_scores_rating.isna(), 1),
    review_scores_rating=data.review_scores_rating.fillna(
        np.median(data.review_scores_rating.dropna())
    ),
    flag_reviews_per_month=np.multiply(data.reviews_per_month.isna(), 1),
    reviews_per_month=data.reviews_per_month.fillna(
        np.median(data.reviews_per_month.dropna())),
    flag_host_response_time=np.multiply(data.host_response_time.isna(), 1),
    host_response_time=data.host_response_time.fillna("a few days or more")
)

In [27]:
data = data.assign(
    ln_days_since=lambda x: np.log(x["n_days_since"] + 1),
    ln_days_since2=lambda x: np.log(x["n_days_since"] + 1) ** 2,
    ln_days_since3=lambda x: np.log(x["n_days_since"] + 1) ** 3,
    n_days_since2=lambda x: x["n_days_since"] ** 2,
    n_days_since3=lambda x: x["n_days_since"] ** 3,
    ln_review_scores_rating=lambda x: np.log(x["review_scores_rating"]+1),
    ln_host_total_listings_count=lambda x: np.log(x["host_total_listings_count"] + 1),
    ln_minimum_nights=lambda x: np.log(x["minimum_minimum_nights"] + 1),
    ln_number_of_reviews=lambda x: np.log(x["number_of_reviews"] + 1),
    ln_reviews_per_month=lambda x: np.log(x["reviews_per_month"] + 1),
    ln_number_of_reviews_l30d=lambda x: np.log(x["number_of_reviews_l30d"] + 1),
)

In [28]:
data.to_csv(Data + "\\airbnb_newyork.csv", index=False)